In [96]:
import librosa as lr
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd
from matrixprofile import matrixProfile
import ffmpeg
from sklearn.preprocessing import normalize
import mass_ts as mts
from scipy.fft import fft, ifft
import numpy as np
import soundfile as sf
from pydub import AudioSegment

In [51]:
def extract_audio(vfile):
    stream = ffmpeg.input(vfile)
    out = ffmpeg.output(stream.audio, 'out.mp3')
    out.run(overwrite_output=True)

def clean_track(track, samp = 22050//4):
    ff_track = fft(track)
    freq_track = np.fft.fftfreq(len(track), d=1/samp)
    ff_track[(freq_track >= 4000) | (freq_track < 300)] = 0
    recon_track = np.real(ifft(ff_track))
    return recon_track

def sim(main, query, sr = 22050//4):
#     batch_sz = sr * 10
    batch_sz = len(main) // 10
#     idxs, dists = mts.mass2_batch(main, query, batch_size=len(query), top_matches=len(main) // len(query) - 1, n_jobs=16)
    idxs, dists = mts.mass2_batch(main, query, batch_size=batch_sz, top_matches=len(main) // batch_sz - 1, n_jobs=16)
    return idxs[np.argsort(dists)]

In [52]:
extract_audio('samples/v2/zoom_0.mp4')

In [53]:
x, sr = lr.core.load('out.mp3', sr=22050//4)
good_x, good_sr = lr.core.load('samples/v2/long_good_audio.mp3', sr=sr)

/home/will/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/will/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [54]:
recon_good = clean_track(good_x)
recon_x = clean_track(x)

In [62]:
idxs = sim(recon_good, recon_x[40*sr:45*sr])

In [63]:
idxs / sr

array([ 54.12282293, 114.8392598 ,  69.23838897, 104.29281567,
        32.45627721,  87.90112482,   0.43196662,  80.72750363,
        43.35921626])

In [64]:
idxs

array([298325, 632994, 381642, 574862, 178899, 484511,   2381, 444970,
       238996])

In [68]:
ipd.Audio(x[40*sr:], rate=sr)

In [67]:
ipd.Audio(good_x[298325:], rate=sr)

In [69]:
offset = idxs[0] - 40*sr

In [72]:
offset / sr

14.122822931785196

In [73]:
x_dur = len(x) / sr

In [75]:
clean_x, clean_sr = lr.core.load('samples/v2/long_good_audio.mp3', sr=48000)

/home/will/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [83]:
start_clean = int(np.round(offset / sr * clean_sr))

In [86]:
end_clean = start_clean + int(np.round(x_dur * clean_sr))

In [89]:
clean_trimmed = clean_x[start_clean:end_clean + 1]

In [94]:
sf.write('fixed.wav', clean_trimmed, clean_sr, 'PCM_16')

In [92]:
sf.default_subtype('WAV')

'PCM_16'

In [80]:
len(clean_x)

6001153

In [31]:
len(good_x) / sr

125.02412917271408

In [97]:
AudioSegment.from_wav('fixed.wav').export('fixed.mp3')

<_io.BufferedRandom name='fixed.mp3'>

In [99]:
vfile = 'samples/v2/zoom_0.mp4'
stream = ffmpeg.input(vfile)
stream2 = ffmpeg.input('fixed.wav')
out = ffmpeg.output(stream.video, stream2.audio, 'fixed.mp4')
out.run(overwrite_output=True)

(None, None)